# *ETL para el archivo user_items*

importamos las librerias para nuestra limpieza

In [5]:
import ast
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import json
import warnings
warnings.filterwarnings('ignore')

In [6]:
#Crear una lista vacía llamada "rows" donde almacenaremos los datos del archivo JSON.
rows = []
#Abrir el archivo "user_reviews.json/australian_user_reviews.json" con la codificación MacRoman.
with open("D:/HENRY\ProyectoPIMLops/PIMLops-STEAM/JSON/australian_users_items.json", encoding='MacRoman') as f:
    # Leer cada línea del archivo.
    for line in f.readlines():
        # Utilizar "ast.literal_eval" para convertir cada línea en un diccionario de Python
        # y agregarlo a la lista "rows".
        rows.append(ast.literal_eval(line))
#Crear un DataFrame de Pandas a partir de la lista de diccionarios "rows".
df_items = pd.DataFrame(rows)

desanidamos "items"

In [7]:
# Desanidamos la columna 'items'
df_items_clean = pd.json_normalize(df_items['items'].explode())

In [8]:
# Ahora unimos el DataFrame expandido con el original
df_items = df_items.drop('items', axis=1)
df_items= df_items_clean.join(df_items)

Eliminamos los nulos y los datos donde "user_id" es nulo e "items_count" significa que no posee juegos

In [9]:
# Eliminar las filas donde todas las columnas son nulas
df_items = df_items.dropna(how='all')
# Eliminar las filas donde 'user_id' o 'items_count' son nulos
df_items= df_items.dropna(subset=['user_id', 'items_count'])

In [10]:
df_items = df_items.dropna(subset=['items_count'])

Eliminamos a los usuarios que no poseen juegos, ya que no aportan datos relevantes a nuestro análisis

In [11]:
# Elimina los usuarios que no poseen juegos (items_count igual a 0)
df_items = df_items[df_items['items_count'] > 0]

In [12]:
df_items.isna().sum()

item_id             112
item_name           112
playtime_forever    112
playtime_2weeks     112
user_id               0
items_count           0
steam_id              0
user_url              0
dtype: int64

# Procedemos a eliminar las columnas que no son relevantes para nuestro futuro modelo

In [13]:
df_items.drop(columns=["playtime_2weeks","user_url","steam_id"], inplace=True)

Eliminamos los nulos ya que son irrelevantes, son usuarios que tienen juegos pero no jugaron nunca, y no hay informacion del juego

In [14]:
df_items.dropna(inplace=True)

### realizamos la siguiente transformacion:

*Observando nuestro modelo, en nuestra funcion estábamos consumiendo mucha memoria para render, con lo cual decidimos pasar la división de "Playtime_forever" sobre 60 para tener horas, dentro de éste ETL*

In [15]:
def convert_playtime_to_hours(df_items):
    df_items['playtime_forever'] = df_items['playtime_forever'] / 60
    return df_items
df_items = convert_playtime_to_hours(df_items)

Realizamos la conversión de tipos de dato para nuestro futuro modelo / eda

In [16]:
df_items['item_id'] = df_items['item_id'].astype(int)
df_items = df_items.astype({'item_name': 'string'})

In [17]:
#eliminamos a los usuarios que no tengan tiempo de juego 
df_items = df_items[df_items['playtime_forever'] != 0]

In [18]:
df_items.to_parquet('user_items.parquet')